# Implicit solvation in BigDFT

**Prerequisite**
Before this notebook, you should:

* Be familiar with Python;
* Understand how Jupyter Notebooks work;
* Successfully complete previous PyBigDFT [Tutorials](https://l_sim.gitlab.io/bigdft-suite/users/guide.html#tutorials), expecially the one on `Building Systems Programmatically` and `Running The Code`.

**Goal** At the end of the present tutorial you will be familiar with the implicit solvation feature of BigDFT for the treatment of:
* an isolated molecule plunged in a solvent;
* a solid/liquid interface.

**Exercises** We propose two exercises:
* The calculation of the solvation free energy of a mesitylene molecule plunged in mesytilene;
* The adsorption energy of H$_2$O at the solid/liquid interface of the diamond (010) termination and water.

**Overview**

First we resume the soft-sphere implicit solvation model. Then we calculate the solvation free energy of two isolated molecules, that are a water and ethanol molecule plunged in water and ethanol solvents, respectively. Finally we calculate the adsorption energy of an O$_2$ molecule at the solid/liquid interface of the Si (010) termination and water.

# The soft-sphere model

BigDFT allows to include solvent effects in an implicit way at the level of Kohn-Sham density functional theory for electronic-structure calculations. Implicit solvation is handled by the _soft-sphere_ continuum solvation model. The soft-sphere model and its parametrization are reported in Ref. [Fisicaro et al. JCTC 2017](https://pubs.acs.org/doi/full/10.1021/acs.jctc.7b00375). 

An explicit/implicit treatment of an atomistic system in contact with a wet environment requires three main ingredients:

* A three-dimensional dielectric cavity represented by a proper function $\epsilon( \textbf{r})$ mimicking the surrounding solvent of a solute as a continuum dielectric; 

* A solver for the generalized Poisson equation [[Fisicaro et al. JCP 2016](https://aip.scitation.org/doi/10.1063/1.4939125)]
$$
\label{gpe}
\nabla \cdot \epsilon( \textbf{r}) \nabla \phi(\textbf{r}) = -4 \pi \rho(\textbf{r}), \tag{1}
$$
where $\phi(\textbf{r})$ is the potential generated by a given charge density $\rho(\textbf{r})$;

* A model for the non-electrostatic terms to the total free energy of solvation.

Within this approach, the interface between the quantum-mechanical solute and the surrounding environment is described by a fully continuous and differentiable permittivity $\epsilon( \textbf{r})$, which is a function of the atomic coordinates. $\epsilon( \textbf{r})$ acquires a value of 1 where the solute system is placed (and where a quantum mechanical approach is applied) and that of the bulk dielectric constant of the surrounding solvent outside.
We define the soft-sphere cavity by means of atomic-centered interlocking spheres described by continuous and differentiable _h_ functions which smoothly switch from 0 inside the sphere to the value 1 outside. Their product, which is still continuous and differentiable in the whole domain, reproduces the analytic cavity.
The continuous and differentiable cavity allows to compute analytically the additional terms to the forces as well as the cavity-dependent non-electrostatic contributions to the total energy. As a consequence, we could obtain very high accuracy on energies and forces.

Our implementation has a small numerical overhead with respect to a gas-phase calculation, thanks to an efficient solver for Eq. \ref{gpe} which relies on a preconditioned conjugate gradient (PCG) algorithm [[Fisicaro et al. JCP 2016](https://aip.scitation.org/doi/10.1063/1.4939125)]. Its preconditioner, based on the BigDFT vacuum Poisson solver [[Genovese et al. JPC 2006](http://dx.doi.org/10.1063/1.2335442)], allows to solve the electrostatic problem in some ten iterations with an exact treatment of free, wire, surface and periodic boundary conditions.

Solvation describes the interaction of a solvent with dissolved molecules or bulk terminations. The electrostatic solvation energy is defined as the difference between the total energy of a given atomic system in the presence of the dielectric environment $G^{el}$ and the energy of the same system in vacuum $G^0$:

$$
\label{elec}
\Delta G^{el} = G^{el} − G^0 . \tag{2}
$$

A full comparison with experimental solvation energies needs the inclusion of nonelectrostatic contributions to obtain the total free energy of solvation $\Delta G^{sol}$. In BigDFT we model the nonelectrostatic terms (the ones related to the cavitation, the repulsion and the dissociation energies) as linear functions of the _quantum surface S_ and
_quantum volume V_ of the dielectric cavity:

$$
\label{nonelec}
\Delta G^{sol} = \Delta G^{el} + (\alpha + \gamma)S + \beta V . \tag{3}
$$

Here $\gamma$ represents the experimental surface tension of the solvent, $\alpha$ and $\beta$ are solvent-specific parameters which can be fitted to experimental data such as solvation free energies of neutral molecules and ions. See Sec. 2 of [Fisicaro et al. JCTC 2017](https://pubs.acs.org/doi/full/10.1021/acs.jctc.7b00375) for more details.
When implicit solvation is activated in a DFT single point energy and force evaluation with BigDFT, the total Kohn-Sham energy $E_{\text{KS}}$ includes both the electrostatic solvation energy, extracting the electrostatic potential from Eq. \ref{gpe}, and the nonelectrostatic contributions following Eq. \ref{nonelec}.
Therefore the solvation free energy a molecule plunged in a solvent read

$$
\label{sol_ene}
\Delta G^{sol} = E^{solvent}_{\text{KS}}(\text{molecule}) - E^{vacuum}_{\text{KS}}(\text{molecule}) , \tag{4}
$$

being $E^{solvent}_{\text{KS}}(\text{molecule})$ and $E^{vacuum}_{\text{KS}}(\text{molecule})$ the BigDFT total Kohn-Sham energy calculated in vacuum and with implicit solvent.

# Solvation free energy of a molecule

Following we employ the implicit solvation features of BigDFT to calculate the free energy of solvation of two isolated molecules, that are a water and ethanol molecule, plunged in water and ethanol solvents, respectively.

## Setting a BigDFT calculator and a vacuum input file

First we settle a BigDFT calculator and a vacuum input file which will be used through the entire section. The implicit solvation features will be activate afterwards by proper input actions which handle and add a new field in the `inp` dictionary.

In [1]:
from BigDFT import Inputfiles as I
from BigDFT.Calculators import SystemCalculator
from BigDFT.Database import Molecules as mols
from futile.Utils import write
import copy

We offer two possibility to run BigDFT. One faster using less memory and a reduced number of geometry optimization steps, another more accurate and with a tight convergence criteria for the structural relaxation.

In [2]:
inp=I.Inputfile()
inp.set_xc('PBE')
inp.set_psp_nlcc()
inp.set_rmult([6.0,10.0])

# Go fast
#inp.set_hgrid(0.5)
#inp.optimize_geometry(method="SQNM", betax=1.0)

# Go accurate
inp.set_hgrid(0.4)
inp.optimize_geometry(method="SQNM", nsteps=500, betax=1.0, frac_fluct=3.0, forcemax=1.0E-4)

In [3]:
code=SystemCalculator(verbose=False, skip=True)

## Water molecule in implicit water

We start with a geometry optimization of an H$_2$O water molecule in vacuum.

In [4]:
sys_name = 'H2O'
H2O=code.run(input=inp, name=sys_name+'-vacuum',posinp=mols.Molecule(sys_name), run_dir="work-solvent")
write('Energy: {} Ha'.format(H2O.energy))

Found 9 different runs
Energy: -17.608960018485412 Ha


Then we optimize the H$_2$O molecule in a water environment described by the soft-sphere implicit solvation model.
Here we trigger an implicit solvation calculation in the `inp` dictionary.

In [5]:
import copy
inpw=copy.deepcopy(inp)
inpw.set_implicit_solvent(solvent='water',minres=1.0e-8)
print(inpw['psolver'])
# 'minres': 0.0001 in inp['psolver']['environment']
# This is anoother important paramter to be set in inp['psolver']['environment'] for the accuracy/speedup balance.
# Values higher than the default 1.e-8 can speedup the simulation at a cost of a reduced accuracy.

{'environment': {'cavity': 'soft-sphere', 'itermax': 20, 'minres': 1e-08, 'fact_rigid': 1.18, 'delta': 0.625, 'gammaS': 72.0, 'alphaS': -60.5, 'betaV': 0.0}}


In [6]:
H2Ow=code.run(input=inpw, name=sys_name+'-solvent',posinp=mols.Molecule(sys_name), run_dir="work-solvent")
write('Energy: {} Ha'.format(H2Ow.energy))

Found 10 different runs
Energy: -17.61900569689985 Ha


Here we calculate the solvation free energy of a water molecule in water following Eq. 4.

In [7]:
from ase import units
Ha_to_kcal_mol= units.Ha * units.mol / units.kcal

ene_solv = H2Ow.energy - H2O.energy

print('Solvation energy of a water molecule in water: {} kcal/mol'.format("%.2f" % (ene_solv * Ha_to_kcal_mol)))

Solvation energy of a water molecule in water: -6.30 kcal/mol


The calculated value has to be compared with the experimental solvation free energy of a H$_2$O molecule in a water environment that is -6.31 kcal/mol

## Ethanol molecule in implicit ethanol

Now we repeat the previous workflow for the calculation of the solvation free energy of an ethanol C$_2$H$_6$O molecule plunged in the ethanol solvent. In this case, the experimental free energy of solvation of C$_2$H$_6$O in an ethanol environment is -5.04 kcal/mol.

In [8]:
sys_name = 'C2H6O'
eta=code.run(input=inp, name=sys_name+'-vacuum',posinp=mols.Molecule(sys_name), run_dir="work-solvent")
write('Energy: {} Ha'.format(eta.energy))
write()

inpe=copy.deepcopy(inp)
inpe.set_implicit_solvent(solvent='ethanol',minres=1.0e-8)
print(inpe['psolver'])
print()

etae=code.run(input=inpe, name=sys_name+'-solvent',posinp=mols.Molecule(sys_name), run_dir="work-solvent")
write('Energy: {} Ha'.format(etae.energy))
print()

ene_solv = etae.energy - eta.energy

print('Solvation energy of an ethanol molecule in ethanol: {} kcal/mol'.format("%.2f" % (ene_solv * Ha_to_kcal_mol)))

Found 30 different runs
Energy: -33.131393892091936 Ha

{'environment': {'cavity': 'soft-sphere', 'itermax': 20, 'minres': 1e-08, 'epsilon': 24.852, 'fact_rigid': 1.26, 'delta': 0.625, 'gammaS': 22.1, 'alphaS': -26.1, 'betaV': 0.0}}

Found 39 different runs
Energy: -33.13816769312041 Ha

Solvation energy of an ethanol molecule in ethanol: -4.25 kcal/mol


## Exercise: Mesitylene molecule in implicit mesitylene

Calculate the solvation free energy of a mesitylene C$_9$H$_{12}$ molecule plunged in mesityle. The experimental solvation energy in this case is -6.40 kcal/mol.

### Solution

In [ ]:
sys_name = 'C9H12'
mesi=code.run(input=inp, name=sys_name+'-vacuum',posinp=mols.Molecule(sys_name), run_dir="work-solvent")
write('Energy: {} Ha'.format(mesi.energy))
print()

inpm=copy.deepcopy(inp)
inpm.set_implicit_solvent(solvent='mesitylene',minres=1.0e-8)
print(inpm['psolver'])
print()

mesim=code.run(input=inpm, name=sys_name+'-solvent',posinp=mols.Molecule(sys_name), run_dir="work-solvent")
write('Energy: {} Ha'.format(mesim.energy))
print()

ene_solv = mesim.energy - mesi.energy

print('Solvation energy of a mesitylene molecule in mesitylene: {} kcal/mol'.format("%.2f" % (ene_solv * Ha_to_kcal_mol)))

# Adsorption energy at a solid/liquid interface

Now we explore the use of the implicit solvation features of BigDFT to model a solid/liquid interface and its interaction with an adsorbed molecule. In particular, we want to explore the adsorption process of an O$_2$ molecule at the solid/liquid interface of the Si (010) termination and water. This represents just an example. The present workflow can be integrated with additional steps to change/modify the adsorbed molecule, the exposed surface at the vacuum and solvent environment, the contact solvent, the molecules/surface configuration etc.

To quantify the interaction between an adsorbed molecule and a bulk termination, we can calculated the adsorption energy as

$$
E_{\text{ads}} = E_{\text{KS}}(\text{molecule}) + E_{\text{KS}}(\text{surface}) - E_{\text{KS}}(\text{molecule}/\text{surface}) \tag{5}
$$

where $E_{\text{KS}}(\text{molecule})$, $E_{\text{KS}}(\text{surface})$ and $E_{\text{KS}}(\text{molecule}/\text{surface})$ represent the total Kohn-Sham energy of an isolated molecule, the surface, and the molecule/surface system, respectively.
The reference systems to compute adsorption energies in vacuum are a slab, an isolated molecule and a slab with an adsorbed molecule in vacuum. 
The reference systems to compute adsorption energies of a molecule at a solid/liquid interface are a slab in contact with the implicit solvent, a molecule immersed in implicit solvent and a slab/liquid interface (liquid described by the implicit model) with an adsorbed molecule.
A positive adsorption energy means that the adsorption is exothermic. All energetics refer to the final relaxed structures in the proper environment, that is vacuum or in the presence of implicit solvent.

When solvent effects are considered at a solid/liquid surface, one surface is considered to be in contact with the implicit
solvent. When adsorption is investigated, this is the surface where the solvent molecules are eventually attached.
For the opposite (bottom) surface, vacuum conditions are always used by setting a large radius of the soft-sphere model for bottom atoms. This guarantees that bottom atoms are never in contact with the implicit solvent.

BigDFT and the soft-spheres implicit solvation model has been already applied for the study of molecular adsorption at various solid/liquid interfaces:
* [Surface reconstruction of fluorites in vacuum and aqueous environment](https://journals.aps.org/prmaterials/abstract/10.1103/PhysRevMaterials.1.033609)
* [Direct observation of single organic molecules grafted on the surface of a silicon nanowire](https://www.nature.com/articles/s41598-019-42073-5)
* [Wet Environment Effects for Ethanol and Water Adsorption on Anatase TiO$_2$ (101) Surfaces](https://pubs.acs.org/doi/abs/10.1021/acs.jpcc.9b05400)

## O$_2$ adsorption at the Si (010) / water interface

First we build a silicon (010) slab with a diamond structure using the [ase](https://wiki.fysik.dtu.dk/ase/_modules/ase/build/bulk.html) package.

In [9]:
from ase.build import bulk

atoms = bulk('Si', 'diamond', 5.431, cubic=True)
atoms *= [1, 2, 1]

In [10]:
from BigDFT.Interop.ASEInterop import ase_to_bigdft
from BigDFT.Systems import System
from BigDFT.UnitCells import UnitCell
slab = System()
slab["SUR:1"] = ase_to_bigdft(atoms)
slab.cell = UnitCell([float(atoms.cell[0, 0]), float("inf"), float(atoms.cell[2, 2])], units="angstroem")

In [11]:
from BigDFT.Visualization import InlineVisualizer
viz = InlineVisualizer(400, 300)
viz.display_system(slab)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Atoms at the bottom (non-hydrated) layer of the slab were fixed at their bulk coordinates to emulate bulk behaviour.
First we identify such atoms to be frozen. Then we gather them in a Fragment.

In [12]:
print('SLAB BEFORE THE FROZEN STEP')
for fragid, frag in slab.items():
    print(fragid)
    for at in frag:
        print(dict(at))

from BigDFT.Fragments import Fragment
from copy import deepcopy
frag = deepcopy(slab["SUR:1"])
y = [dict(at)['r'][1] for at in frag]
slab["SUR:2"] = Fragment()
for at in frag:
    if dict(at)['r'][1] < min(y) + 0.5:
        print(dict(at))
        slab["SUR:2"].append(at)
        slab["SUR:1"].remove(at)

print()
print('SLAB AFTER THE FROZEN STEP')
for fragid, frag in slab.items():
    print(fragid)
    for at in frag:
        print(dict(at))

SLAB BEFORE THE FROZEN STEP
SUR:1
{'sym': 'Si', 'r': array([0., 0., 0.]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 1.35775, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.    , 2.7155, 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 4.07325, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 0.    , 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 1.35775, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 2.7155, 0.    ]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 4.07325, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.   , 5.431, 0.   ]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 6.78875, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.    , 8.1465, 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 9.50425, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 5.431 , 2.7155]), 'units': 'angstroem'}
{

The Fragment to be frozen is highlighted by the BigDFT `InlineVisualizer` which differentiates different fragments with different colours. The visualization can support the choice/tuning of the slab layers to be frozen.

In [13]:
from BigDFT.Visualization import InlineVisualizer
viz = InlineVisualizer(400, 300)
viz.display_system(slab)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we directly `Frozen` the entire Fragment with a single Input Action.

In [14]:
slab["SUR:2"].frozen = "fxyz"

Now we are ready to create an O$_2$ molecule and to attach it to the silicon (010) slab.

In [15]:
from BigDFT.IO import XYZReader
mol = System()
with XYZReader("O2") as ifile:
    mol["ABS:2"] = Fragment(xyzfile=ifile)

slab_mol = deepcopy(slab)
slab_mol["ABS:2"] = deepcopy(mol["ABS:2"])
slab_mol["ABS:2"].translate([x - y for x, y in zip(slab_mol["SUR:1"].centroid, slab_mol["ABS:2"].centroid)])
slab_mol["ABS:2"].translate([0, slab_mol.cell[2, 2], 0])

In [16]:
for fragid, frag in slab_mol.items():
    print(fragid)
    for at in frag:
        print(dict(at))

SUR:1
{'sym': 'Si', 'r': array([1.35775, 1.35775, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.    , 2.7155, 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 4.07325, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 1.35775, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 2.7155, 0.    ]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 4.07325, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.   , 5.431, 0.   ]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 6.78875, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.    , 8.1465, 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 9.50425, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 5.431 , 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 6.78875, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 8.1465, 0.    ]), 'units': 'angstroem'}
{'sym': 'Si', 

In [17]:
from BigDFT.Visualization import InlineVisualizer
viz = InlineVisualizer(400, 300)
viz.display_system(slab_mol)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we settle a BigDFT vacuum input file.

In [18]:
inp = I.Inputfile()
inp.set_xc('PBE')
inp.set_psp_nlcc()
inp.set_hgrid('0.5')
inp.optimize_geometry(method="SQNM", betax=1.0)

Following Eq. 5 we calculte the adsoption energy of the O$_2$ molecule at the silicon (010) surface in vacuum.

In [19]:
from ase import units

# calculation in vacuum
sys_mol=code.run(input=inp, name='mol-vacuum',posinp=mol.get_posinp(), run_dir="work-solvent")
write('Energy: {} Ha'.format(sys_mol.energy))

sys_slab=code.run(input=inp, name='slab-vacuum',posinp=slab.get_posinp(), run_dir="work-solvent")
write('Energy: {} Ha'.format(sys_slab.energy))

sys_slab_mol=code.run(input=inp, name='slab-mol-vacuum',posinp=slab_mol.get_posinp(), run_dir="work-solvent")
write('Energy: {} Ha'.format(sys_slab_mol.energy))

ads_ene = sys_slab.energy + sys_mol.energy - sys_slab_mol.energy

print('Adsorption energy in vaccum: {} eV'.format(ads_ene * units.Ha))

Found 25 different runs
Energy: -32.62842714026817 Ha
Found 51 different runs
Energy: -114.63083393631985 Ha
Found 51 different runs
Energy: -145.28036372617885 Ha
Adsorption energy in vaccum: -53.848539704581455 eV


Here we trigger an implicit solvation calculation in the `inp` dictionary. The water environment is described by the soft-sphere implicit solvation model.

In [20]:
import copy
inpw=copy.deepcopy(inp)
inpw.set_implicit_solvent(solvent='water',minres=1.0e-8)
print(inpw['psolver'])

{'environment': {'cavity': 'soft-sphere', 'itermax': 20, 'minres': 1e-08, 'fact_rigid': 1.18, 'delta': 0.625, 'gammaS': 72.0, 'alphaS': -60.5, 'betaV': 0.0}}


Before to submit an implicit solvation run for a solid/liquid interface, it is important to check the quality of the dielectric cavity $\epsilon( \textbf{r})$. Depending on the radii of the element-dependent soft-spheres and the distance between neareast neighbors atoms, a side effect of the implicit solvation model could emerge with small unphysical pocket of implicit solvent, smaller then a solvent molecule, within the bulk and far from the solid/liquid interface.
A fast solution can be to artificially enlarge all soft-spheres radii to a fixed $r_{bulk}$ value for bulk atoms which lies below a $\delta y$ from the solid/liquid interface. An $r_{bulk} = 3.0 \mathring A $ and $\delta y = 3.0 \mathring A $ guarantee an exclusion of implicit solvent inside the bulk system.
These atoms are first identified and then gathered in a Fragment. As previously done, the fragment splitting of the entire slab can be easily visualized for a visual inspection.

In [21]:
print('SLAB BEFORE THE CAVITY STEP')
for fragid, frag in slab.items():
    print(fragid)
    for at in frag:
        print(dict(at))

print()
from BigDFT.Fragments import Fragment
from copy import deepcopy
frag = deepcopy(slab["SUR:1"])
y = [dict(at)['r'][1] for at in frag]
slab["SUR:3"] = Fragment()
for at in frag:
    if dict(at)['r'][1] < max(y) - 3.0:
        print(dict(at))
        slab["SUR:3"].append(at)
        slab["SUR:1"].remove(at)

print()
print('SLAB AFTER THE CAVITY STEP')
for fragid, frag in slab.items():
    print(fragid)
    for at in frag:
        print(dict(at))
        
from BigDFT.Visualization import InlineVisualizer
viz = InlineVisualizer(400, 300)
viz.display_system(slab)

SLAB BEFORE THE CAVITY STEP
SUR:1
{'sym': 'Si', 'r': array([1.35775, 1.35775, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.    , 2.7155, 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 4.07325, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 1.35775, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 2.7155, 0.    ]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 4.07325, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.   , 5.431, 0.   ]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 6.78875, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.    , 8.1465, 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 9.50425, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 5.431 , 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 6.78875, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 8.1465, 0.    ]), 'units':

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [22]:
print('SLAB BEFORE THE CAVITY STEP')
for fragid, frag in slab_mol.items():
    print(fragid)
    for at in frag:
        print(dict(at))

print()
from BigDFT.Fragments import Fragment
from copy import deepcopy
frag = deepcopy(slab_mol["SUR:1"])
y = [dict(at)['r'][1] for at in frag]
slab_mol["SUR:3"] = Fragment()
for at in frag:
    if dict(at)['r'][1] < max(y) - 3.0:
        print(dict(at))
        slab_mol["SUR:3"].append(at)
        slab_mol["SUR:1"].remove(at)

print()
print('SLAB AFTER THE FROZEN STEP')
for fragid, frag in slab_mol.items():
    print(fragid)
    for at in frag:
        print(dict(at))
        
from BigDFT.Visualization import InlineVisualizer
viz = InlineVisualizer(400, 300)
viz.display_system(slab_mol)

SLAB BEFORE THE CAVITY STEP
SUR:1
{'sym': 'Si', 'r': array([1.35775, 1.35775, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.    , 2.7155, 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 4.07325, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 1.35775, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 2.7155, 0.    ]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 4.07325, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.   , 5.431, 0.   ]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 6.78875, 1.35775]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([0.    , 8.1465, 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([1.35775, 9.50425, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 5.431 , 2.7155]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([4.07325, 6.78875, 4.07325]), 'units': 'angstroem'}
{'sym': 'Si', 'r': array([2.7155, 8.1465, 0.    ]), 'units':

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we modify the soft-spheres radius to a fixed $r_{bulk}$ value for the entire Fragment with a single Input Action.

In [23]:
for at in slab["SUR:2"]:
    at.update({'rcav': 3.0})
for at in slab["SUR:3"]:
    at.update({'rcav': 3.0})
for at in slab_mol["SUR:2"]:
    at.update({'rcav': 3.0})
for at in slab_mol["SUR:3"]:
    at.update({'rcav': 3.0})

Following Eq. 5 we calculate the adsoption energy of the O$_2$ molecule at the silicon (010) surface in contact with a water environment.

In [24]:
# calculation in vacuum
sys_mol=code.run(input=inpw, name='mol-solvent',posinp=mol.get_posinp(), run_dir="work-solvent")
write('Energy: {} Ha'.format(sys_mol.energy))

sys_slab=code.run(input=inpw, name='slab-solvent',posinp=slab.get_posinp(), run_dir="work-solvent")
write('Energy: {} Ha'.format(sys_slab.energy))

sys_slab_mol=code.run(input=inpw, name='slab-mol-solvent',posinp=slab_mol.get_posinp(), run_dir="work-solvent")
write('Energy',sys_slab_mol.energy)
write('Energy: {} Ha'.format(sys_slab_mol.energy))

ads_ene = sys_slab.energy + sys_mol.energy - sys_slab_mol.energy

print('Adsorption energy in water: {} eV'.format(ads_ene * units.Ha))

Found 22 different runs
Energy: -32.6288477068833 Ha
Found 45 different runs
Energy: -108.31461168187421 Ha
Found 51 different runs
Energy -145.31996209529638
Energy: -145.31996209529638 Ha
Adsorption energy in water: 119.09070458431728 eV


## Exercise: Water adsorption at (010) diamond surface

Following a similar workflow previously described, calculate the adsorption energy of a water H$_2$O molecule at the solid/liquid interface of the diamond (010) termination and water.